In [ ]:
import os

import pandas as pd

# Parameters

In [ ]:
database_files = snakemake.input.database_files
enrichment_files = snakemake.input.enrichment_files

fname_data = snakemake.output.fname_data
fname_enr = snakemake.output.fname_enr

# Load data

In [ ]:
df_list = []
for fname in database_files:
    tmp = pd.read_csv(fname)
    
    parts = os.path.basename(fname).split('.')
    tmp['tad_source'] = parts[1]
    tmp['window_size'] = parts[2]
    tmp['filter'] = parts[3]
    
    df_list.append(tmp)

df_data = pd.concat(df_list)
df_data.sort_values(['diseaseId', 'snpId']).head()

In [ ]:
df_list = []
for fname in enrichment_files:
    tmp = pd.read_csv(fname)
    
    parts = os.path.basename(fname).split('.')
    tmp['tad_source'] = parts[1]
    tmp['window_size'] = parts[2]
    tmp['filter'] = parts[3]
    
    df_list.append(tmp)

df_enr = pd.concat(df_list)
df_enr.head()

# Save results

In [ ]:
df_data.to_csv(fname_data, index=False)

In [ ]:
df_enr.to_csv(fname_enr, index=False)